In [2]:
pip install -q langchain langchain-community langchain_huggingface chromadb

Note: you may need to restart the kernel to use updated packages.


## 1. Use langchain RAG

In [3]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [6]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_API_TOKEN"

In [ ]:
# os.environ.get("HUGGINGFACEHUB_API_TOKEN")

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub

# set Korean embedding and llm odel
hf_embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

hf_llm = HuggingFaceHub(
    repo_id="skt/kogpt2-base-v2",
    model_kwargs={"task": "text-generation"} ## question-answering tasK X. text-generation
)

/tmp/ipykernel_30/3316765938.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipykernel_30/3316765938.py:7: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  hf_llm = HuggingFaceHub(


In [8]:
import requests
from langchain.schema import Document
from bs4 import BeautifulSoup

# for Wikipedia documents (EN, KO)

# from langchain_community.document_loaders import WikipediaLoader

# By default, English documents (https://en.wikipedia.org))
# def load_Wiki_docs(query):
#     loader = WikipediaLoader(query=query, load_max_docs=1) # need !pip install wikipedia
#     documents = loader.load()
    
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=1000,
#         chunk_overlap=200
#     )
#     splits = text_splitter.split_documents(documents)
    
#     return splits


# For Korean query, get results from Korean wikipedia website and crawl and parse results
def load_Korean_wiki_docs(topic):
    url = f"https://ko.wikipedia.org/wiki/{topic}"
    
    response = requests.get(url)
    response.raise_for_status()  # raise Exception when error occurs

    # HTML parsing and extract body contents
    soup = BeautifulSoup(response.text, 'html.parser')
    content = soup.find('div', {'class': 'mw-parser-output'})  # find div including body contents 
    
    # Extract contents
    paragraphs = content.find_all('p')
    text = "\n".join([p.get_text() for p in paragraphs])  # concat all context in <p> tags 
 
    # convert to Document object (required for LangChain)
    documents = [Document(page_content=text, metadata={"source": url})]
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(documents)
    
    return splits

In [9]:
def create_vectorstore(splits): 
    vectorstore = Chroma.from_documents(documents=splits, embedding=hf_embeddings)
    return vectorstore

In [10]:
topic = "흑백요리사"
# Load wikipedia documents for this topic
splits = load_Korean_wiki_docs(topic) 
# Create vectorstore with this fetched docs
vectorstore = create_vectorstore(splits)

In [11]:
def create_rag_chain(vectorstore):
    prompt_template = """문맥을 참고하여 질문에 정확하고 간결하게 답하십시오.
    문맥: {context}
    질문: {question}
    답변:"""
    
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    chain_type_kwargs = {"prompt": PROMPT}

    # Make context shorter
    # def short_context(context, max_length=300):
    #     return context[:max_length] if len(context) > max_length else context
    
    # class ShortContextRetriever(BaseRetriever):
    #     def __init__(self, retriever):
    #         super().__init__()
    #         self._retriever = retriever
        
    #     def get_relevant_documents(self, query):
    #         docs = self._retriever.get_relevant_documents(query)
    #         for doc in docs:
    #             doc.page_content = short_context(doc.page_content)
    #         return docs
    
    # retriever = ShortContextRetriever(vectorstore.as_retriever())

    qa_chain = RetrievalQA.from_chain_type(
        llm=hf_llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs=chain_type_kwargs,
        return_source_documents=True
    )
    
    return qa_chain

In [12]:
# create langchang RAG chain
qa_chain = create_rag_chain(vectorstore)

In [13]:
question = "심사위원을 누가 맡았어?"

# result = qa_chain({"query": question})
result = qa_chain.invoke({"query": question})

print ("결과:")
print(result["result"])

print("출처:")
for doc in result["source_documents"]:
    print(doc.page_content)
    print("---")

결과:
문맥을 참고하여 질문에 정확하고 간결하게 답하십시오.
    문맥: 《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]
    질문: 심사위원을 누가 맡았어?
    답변: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이

In [14]:
docs = vectorstore.as_retriever().get_relevant_documents(question)
docs

/tmp/ipykernel_30/2742417880.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = vectorstore.as_retriever().get_relevant_documents(question)


[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]')]

In [15]:
docs = vectorstore.similarity_search(question, k=4)
docs

[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]')]

In [ ]:
# It seems vectorDB loading from embedding model works fine, but seems llm model does not.
# Some Korean llm model seems to work fine in text-generation task, but for Question-Ansering task, we might need another approach.

## 2. Use QA pipeline with vectorstor similarity search

In [16]:
# import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

# Load model and tokenizer
model_name = "yjgwak/klue-bert-base-finetuned-squard-kor-v1"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set Q_A pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

config.json:   0%|          | 0.00/635 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/246k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
# Example: define question and context 
question = "오늘 날씨 어때?"
context = "오늘의 날씨는 맑고 따뜻한 기온이 유지될 것으로 보입니다."

# model chain
result = qa_pipeline(question=question, context=context)

# Result
print("질문:", question)
print("답변:", result['answer'])

In [ ]:
# search context in VectorStore
def retrieve_context(question, vectorstore):
    docs = vectorstore.similarity_search(question, k=4)
    if docs:
        return " ".join([doc.page_content for doc in docs])
        # return docs[0].page_content  # return first relevant doc
    else:
        return None

# Generate answer based on query and searched context similar to RAG chain
def answer_question_with_context(question, vectorstore):
    context = retrieve_context(question, vectorstore)
    if context:
        result = qa_pipeline(question=question, context=context)
        return result['answer'], context  # return answer and used source doc
    else:
        return "관련 문맥을 찾지 못했습니다.", None

In [ ]:
# Example
question = "심사위원을 누가 맡았어?"

answer, used_context = answer_question_with_context(question, vectorstore)

print("질문:", question)
print("답변:", answer)
print("사용된 문맥:", used_context)

## 3. Use Gemini+RAG

In [ ]:
# It seems the best and simple and cost-free option when OpenAI api cannot be used.

In [17]:
pip install -q langchain langchain-community langchain_huggingface chromadb google-generativeai

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# pip install google-generativeai

In [18]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.llms import OpenAI
import google.generativeai as genai
import os

In [19]:
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR-API-KEY"
genai_api_key = "your_api_key"

In [20]:
genai.configure(api_key=genai_api_key)

In [21]:
# 1. Gemini model
gemini_model = genai.GenerativeModel('gemini-1.5-flash')

# 2. embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [32]:
from langchain.vectorstores import Chroma
# sample docs
docs = [
    Document(page_content="한국어 챗봇은 자연어 처리 기술을 사용하여 사용자와 대화를 나눕니다.", metadata={"source": "doc1"}),
    Document(page_content="인공지능을 활용한 챗봇은 여러 산업에서 사용되고 있습니다.", metadata={"source": "doc2"}),
    Document(page_content="한국어와 영어를 동시에 지원하는 챗봇이 점점 늘어나고 있습니다.", metadata={"source": "doc3"}),
    Document(page_content="챗봇은 고객 서비스를 개선하고 사용자 경험을 향상시키는 데 중요한 역할을 합니다.", metadata={"source": "doc4"})
]

# to avoid collision with previous one
persist_directory = "./new_chroma_db"

vectorstore = Chroma.from_documents(docs, embedding=embedding_model, persist_directory="./chroma_db")

In [39]:
# RAG using prompt
def rag_chatbot(question):
    context_doc = vectorstore.similarity_search(question, k=1)
    # context = context_doc[0].page_content if context_doc else "정보를 찾을 수 없습니다."

    context = " ".join([doc.page_content for doc in context_doc])
    
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer in a complete sentence:"
    # response = gemini_model(prompt)
    
    response = gemini_model.generate_content(prompt)
    answer = response.candidates[0].content.parts[0].text

    # print("출처 문서:", context)
    return answer

In [40]:
# sample question
question = "챗봇이 어떤 기술을 사용하나요?"
response = rag_chatbot(question)

print("질문:", question)
print("답변:", response)

질문: 챗봇이 어떤 기술을 사용하나요?
답변: 챗봇은 자연어 처리(NLP), 기계 학습(ML), 심층 학습(DL)과 같은 인공지능 기술을 사용합니다.



In [31]:
prompt = f"질문: 김이나 작사가의 작사 특이점으로 적금을 주제로 작사해 줄래? \n대답: "
response = gemini_model.generate_content(prompt)
    
answer = response.candidates[0].content.parts[0].text
print(answer)

## 콩나물 시루 같은 내 적금통장 (김이나풍 가사)

(Verse 1)
월급날 딱 떨어진 알림에 심장이 콩닥
세상 즐거운 소리, 삑! 하고 울리는 경고음 같아
오늘도 난 짠돌이 모드, 텅장의 늪에서 벗어나
저축 버튼 누르는 손길, 왠지 모르게 뿌듯해

(Chorus)
콩나물 시루 같은 내 적금통장, 콩알만 한 희망이 자라나
매달 조금씩 불어나는 숫자, 작지만 확실한 행복의 증거
미래의 나를 위한 선물, 꿈을 향한 작은 발걸음
언젠가 펼쳐질 화려한 날들을 위해, 오늘도 난 저축해

(Verse 2)
치킨 한 마리 포기, 쇼핑 욕구 참아내는 연습
마음은 벌써 백화점, 현실은 적금 이자 계산
친구들 파티 초대, 괜찮아, 난 내일의 나를 위해
오늘 밤은 잔잔한 음악과 함께, 미래를 꿈꿔봐

(Chorus)
콩나물 시루 같은 내 적금통장, 콩알만 한 희망이 자라나
매달 조금씩 불어나는 숫자, 작지만 확실한 행복의 증거
미래의 나를 위한 선물, 꿈을 향한 작은 발걸음
언젠가 펼쳐질 화려한 날들을 위해, 오늘도 난 저축해

(Bridge)
어쩌면 늦은 건 아닐까, 망설이고 고민했던 시간들
하지만 시작이 반이라 했지, 작은 씨앗 하나 심어놓고
묵묵히 기다리는 마음, 그 안에 꿈틀거리는 설렘
이 작은 콩알들이 언젠가 푸른 잎을 낼 거야

(Chorus)
콩나물 시루 같은 내 적금통장, 콩알만 한 희망이 자라나
매달 조금씩 불어나는 숫자, 작지만 확실한 행복의 증거
미래의 나를 위한 선물, 꿈을 향한 작은 발걸음
언젠가 펼쳐질 화려한 날들을 위해, 오늘도 난 저축해

(Outro)
콩나물 시루…  내 통장…  (작은 목소리로)  쑥쑥 자라라…


**김이나 작사가 특징 반영:**

* **일상적인 소재와 감정을 활용:**  적금이라는 평범한 소재를  '콩나물 시루' 와 같은 친근한 비유로 표현.
* **솔직하고 공감가는 가사:**  저축의 어려움과 뿌듯함을 현실적으로 표현.  ('치킨 한 마리 포기', '쇼핑 욕구 참아내는 연습' 등)
* **반복되는 후렴구와 간결한 표현:*